In [1]:
import time

from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from awkward import JaggedArray
import numpy as np
import glob as glob
import itertools
import json
import uproot_methods
import copy

ModuleNotFoundError: No module named 'scipy.interpolate'

In [ ]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class JetMassProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")

        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 
                           np.array([0,5,10,15,20,25,30,35,40,45,50,60,70,80,90,
                                     100,120,140,160,180,
                                     200,250,300,350,400,450,500,
                                     600,700,800,900,1000,
                                     1500,2000,3000,4000,5000]))
        m_axis = hist.Bin("m", r"$p_{T}$ [GeV]", 200, 0, 500)
        dr_axis = hist.Bin("dr", r"$\Delta r$", 80, 0, 0.8)
        
        self._accumulator = processor.dict_accumulator({
            'pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'm':hist.Hist("Counts", dataset_axis, m_axis),
            'pt_v_m':hist.Hist("Counts", dataset_axis, pt_axis, m_axis )
            'dr':hist.Hist("Counts", dataset_axis, dr_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()

        #print("1")
        dataset = df['dataset']
        Jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['FatJet_pt'] * (1 - df['FatJet_rawFactor']),
            eta=df['FatJet_eta'],
            phi=df['FatJet_phi'],
            mass=df['FatJet_mass'],
            jetId=df['FatJet_jetId']
            )        
        GenJets = JaggedCandidateArray.candidatesfromcounts(
            df['nGenJetAK8'],
            pt=df['GenJetAK8_pt'],
            eta=df['GenJetAK8_eta'],
            phi=df['GenJetAK8_phi'],
            mass=df['GenJetAK8_mass']
            )

        evtweights = df["Generator_weight"].reshape(-1, 1).flatten()
        output['cutflow']['all events'] += Jets.size

        jetId_cut = (Jets.jetId > 0)
        Jets = Jets[jetId_cut]
        output['cutflow']['>=1 with loose id'] += jetId_cut.any().sum()        
        
        pairing = Jets.p4.cross(GenJets.p4, nested=True)
        metric = pairing.i0.delta_r(pairing.i1)
        
        index_of_minimized = metric.argmin()
        dr_cut = (metric[index_of_minimized] < 0.8)
        best_pairings_that_pass_dr_cut = pairing[index_of_minimized][dr_cut]
        genrecos = best_pairings_that_pass_dr_cut.flatten(axis=1)
        #print("genrecos shape:", genrecos.shape)
        ptresponse = genrecos.i0.pt / genrecos.i1.pt
        mresponse = genrecos.i0.m / genrecos.i1.m
        msdresponse =  
        
        output['pt'].fill(dataset=dataset,
                            pt=Jets.pt.flatten())
        output['eta'].fill(dataset=dataset, 
                                 eta=Jets.eta.flatten())
        output['r_pt_ptveta'].fill( dataset=dataset, pt=genrecos.i1.pt.flatten(), eta=genrecos.i1.eta.flatten(), r=ptresponse.flatten())
        return output


        
        
        
        return output

    def postprocess(self, accumulator):
        return accumulator


In [ ]:
# Get the sample metadat

with open("DYJetsToLLM-50TuneCUETP8M113TeV-madgraphMLM-pythia8RunIISummer16MiniAODv3-PUMoriond17_ext2-v2.txt") as f:
    filenames = f.readlines().rstrip()
    for fi in filenames:
        fi = 'root://cmsxrootd.fnal.gov//' + fi
        
print(filenames)
fileset = {"DY":filenames}

tstart = time.time() 
output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=FancyJECL2L3Processor(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 4, 'flatten': True},
                                  chunksize=500000,
                                 )


elapsed = time.time() - tstart
print(output)

In [ ]:
#cache1 = {}
#Ttest = uproot.open('/mnt/cms-data/store/group/lpctlbsm/NanoAODJMAR_2019_V1/Production/CRAB/DYJetsToLL_M-50_HT-800to1200_TuneCP5_13TeV-madgraphMLM-pythia8/DYJetsToLLM-50HT-800to1200TuneCP513TeV-madgraphMLM-pythia8RunIIFall17MiniAODv2-PU2017/190312_200548/0000/nano102x_on_mini94x_2017_mc_NANO_9.root')['Events']
#Ttest.arrays("*",cache=cache1)

In [ ]:
# Set fonts (from https://stackoverflow.com/questions/3899980/how-to-change-the-font-size-on-a-matplotlib-plot)
SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [ ]:
#H2 = H.reshape( (len(ptbins)-1)*(len(mbins)-1), (len(ptbins)-1)*(len(mbins)-1) )

In [ ]:
plt.imshow(H2)